In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
import torch.optim as optim
import torch.nn.functional as F
import string
!pip install clean-text
from cleantext import clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=2fc58bf6c6eff81cad6af505a38da5a38ead6b89537b8288579fdf756bb9c701
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.11.0
    Uninstalling emoji-2.11.0:
      Successfully uninstalled emoji-2.11.0


In [2]:
# Load the dataset from Hugging Face
dataset = load_dataset("123rc/medical_text")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
train_data = dataset["train"]
test_data = dataset["test"]

In [4]:
# Convert train and test splits to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [5]:
# Rename two columns 
train_df.rename(columns={'condition_label': 'label', 'medical_abstract': 'text'}, inplace=True)

In [6]:
# Rename two columns 
test_df.rename(columns={'condition_label': 'label', 'medical_abstract': 'text'}, inplace=True)

In [7]:
train_df['label'] = train_df['label'] - 1
test_df['label'] = test_df['label'] - 1

In [8]:
train_df['label'].value_counts()

label
4    3844
0    2530
3    2441
2    1540
1    1195
Name: count, dtype: int64

In [9]:
train_df['label'].nunique()

5

In [10]:
test_df.head()

,label,text
0,2,Obstructive sleep apnea following topical orop...
1,4,Neutrophil function and pyogenic infections in...
2,4,A phase II study of combined methotrexate and ...
3,0,Flow cytometric DNA analysis of parathyroid tu...
4,3,Paraneoplastic vasculitic neuropathy: a treata...


In [11]:
train_df.shape

(11550, 2)

In [12]:
test_df.shape

(2888, 2)

In [13]:
texts = train_df['text'].tolist()
labels = train_df['label'].tolist()

In [14]:
train_texts, val_texts, train_labels, val_labelss = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [15]:
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

In [16]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = [str(text) for text in texts]
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [17]:
class BioBERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BioBERTClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [18]:
# Set up parameters
bert_model_name = 'dmis-lab/biobert-base-cased-v1.2'
num_classes = 5
max_length = 512
batch_size = 8

In [19]:
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labelss, tokenizer, max_length)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BioBERTClassifier(bert_model_name, num_classes).to(device)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [21]:
epochs = 20
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10

# Set the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_dataloader) * epochs // gradient_accumulation_steps
)


In [22]:
import time  # Import the time module

In [23]:
# Start training
start_time = time.time()
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
            optimizer.step()
            scheduler.step()

    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_dataloader)
    train_loss /= len(train_dataloader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1: {f1}, Micro F1: {micro_f1:.4f}, Macro Roc Auc: {macro_roc_auc:.4f}')
         
    # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        
training_time = time.time() - start_time
print(f"\\Total training time: {training_time} seconds")
# Save the model to the current working directory
torch.save(model.state_dict(), "ctc_biobert_base.pth")  
# Move the model to the output directory
# import shutil
# shutil.copy("/kaggle/working/ctc_biobert_pipeline.pth", "/kaggle/outputs/ctc_biobert_pipeline.pth")

Epoch: 1/20, Training Loss: 1.3609853491896675, Validation Loss: 0.9285100366417512
Accuracy: 0.6377, Recall: 0.6377, Precision: 0.6346, F1: 0.6223614550838819, Micro F1: 0.6377, Macro Roc Auc: 0.8781
Epoch: 2/20, Training Loss: 0.8730861813217015, Validation Loss: 0.8322997365443352
Accuracy: 0.6355, Recall: 0.6355, Precision: 0.6309, F1: 0.6241606782968948, Micro F1: 0.6355, Macro Roc Auc: 0.8972
Epoch: 3/20, Training Loss: 0.7772659588556785, Validation Loss: 0.8228822790833905
Accuracy: 0.6403, Recall: 0.6403, Precision: 0.6539, F1: 0.6187326796695815, Micro F1: 0.6403, Macro Roc Auc: 0.9021
Epoch: 4/20, Training Loss: 0.7149761119317183, Validation Loss: 0.8212055966103365
Accuracy: 0.6394, Recall: 0.6394, Precision: 0.6414, F1: 0.6225798450524515, Micro F1: 0.6394, Macro Roc Auc: 0.9026
Epoch: 5/20, Training Loss: 0.6646969760522182, Validation Loss: 0.8412263761548435
Accuracy: 0.6277, Recall: 0.6277, Precision: 0.6257, F1: 0.6137524980613451, Micro F1: 0.6277, Macro Roc Auc: 0.

In [24]:
model.eval()

test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())


In [25]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

test_preds_class = np.argmax(test_preds, axis=1)

report = classification_report(test_labels, test_preds_class, digits = 4)

print(report)

              precision    recall  f1-score   support

           0     0.6944    0.7788    0.7342       633
           1     0.4696    0.7224    0.5692       299
           2     0.5908    0.6000    0.5954       385
           3     0.6570    0.7443    0.6979       610
           4     0.5770    0.3819    0.4596       961

    accuracy                         0.6098      2888
   macro avg     0.5978    0.6455    0.6112      2888
weighted avg     0.6104    0.6098    0.5996      2888

